# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kiunga,-6.1219,141.2906,34.31,50,100,1.91,PG,1683434684
1,1,thompson,55.7435,-97.8558,8.09,61,20,3.09,CA,1683434684
2,2,iqaluit,63.7506,-68.5145,-6.15,85,0,10.29,CA,1683434684
3,3,shahr-e babak,30.1165,55.1186,21.64,12,0,3.09,IR,1683434684
4,4,utrik,11.2278,169.8474,27.39,81,91,7.73,MH,1683434685
...,...,...,...,...,...,...,...,...,...,...
567,567,soritor,-6.1392,-77.1039,19.29,99,75,0.55,PE,1683434805
568,568,pacocha,-17.5833,-71.3333,18.95,86,100,2.96,PE,1683434805
569,569,faro,37.0194,-7.9322,17.32,77,20,3.09,PT,1683434635
570,570,chui,-33.6971,-53.4616,18.85,97,100,3.39,UY,1683434806


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    hover_cols=['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Max Temp'],
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
    size= "Humidity"
)


# Display the map
map_plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df['Max Temp'] >=22) & (city_data_df['Max Temp'] <= 27)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,puerto ayora,-0.7393,-90.3518,26.42,95,100,3.13,EC,1683434685
17,17,cumana,10.4635,-64.1775,25.32,78,96,4.92,VE,1683434687
18,18,maragogi,-9.0122,-35.2225,24.94,81,49,0.92,BR,1683434687
21,21,adamstown,-25.0660,-130.1015,22.95,74,99,6.68,PN,1683434687
24,24,monrovia,6.3005,-10.7969,26.87,80,99,1.00,LR,1683434688
...,...,...,...,...,...,...,...,...,...,...
544,544,saint-joseph,-21.3667,55.6167,25.82,71,22,4.89,RE,1683434801
548,548,mankono,8.0586,-6.1897,23.01,88,100,1.87,CI,1683434802
555,555,naze,28.3667,129.4833,23.51,90,100,8.96,JP,1683434803
566,566,birjand,32.8663,59.2211,22.26,12,0,2.06,IR,1683434805


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,puerto ayora,EC,-0.7393,-90.3518,95,
17,cumana,VE,10.4635,-64.1775,78,
18,maragogi,BR,-9.0122,-35.2225,81,
21,adamstown,PN,-25.0660,-130.1015,74,
24,monrovia,LR,6.3005,-10.7969,80,
...,...,...,...,...,...,...
544,saint-joseph,RE,-21.3667,55.6167,71,
548,mankono,CI,8.0586,-6.1897,88,
555,naze,JP,28.3667,129.4833,90,
566,birjand,IR,32.8663,59.2211,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    #print(name_address.url)
    # Convert the API response to JSON format
    name_address = name_address.json()

    
     # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
cumana - nearest hotel: Hotel Regina
maragogi - nearest hotel: Pousada Mariluz
adamstown - nearest hotel: No hotel found
monrovia - nearest hotel: Bella Casa Hotel
ha'il - nearest hotel: فندق كلاسيك
puerto baquerizo moreno - nearest hotel: Dory's House
bayji - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
sinabang - nearest hotel: No hotel found
bonny - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
slidell - nearest hotel: TownePlace Suites Slidell
portel - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
talara - nearest hotel: Hospedaje El Aypate
tolanaro - nearest hotel: Hôtel Mahavokey
boa vista - nearest hotel: Aipana Plaza Hotel
meulaboh - nearest hotel: Hotel Meuligou
cabo san lucas - nearest hotel: Comfort Rooms
lamu - nearest hotel: Hotel Archip

,City,Country,Lat,Lng,Humidity,Hotel Name
7,puerto ayora,EC,-0.7393,-90.3518,95,Hostal La Mirada De Solitario George
17,cumana,VE,10.4635,-64.1775,78,Hotel Regina
18,maragogi,BR,-9.0122,-35.2225,81,Pousada Mariluz
21,adamstown,PN,-25.0660,-130.1015,74,No hotel found
24,monrovia,LR,6.3005,-10.7969,80,Bella Casa Hotel
...,...,...,...,...,...,...
544,saint-joseph,RE,-21.3667,55.6167,71,PALM Hotel & Spa
548,mankono,CI,8.0586,-6.1897,88,Hôtel Paillotes
555,naze,JP,28.3667,129.4833,90,ビジネス旅館畠山
566,birjand,IR,32.8663,59.2211,12,مهمانسرای جهانگردی بیرجند


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    hover_cols=['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name'],
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
    color = "City"
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)

In [41]:
#SELF BONUS - LIMIT THE MAP TO ONLY SHOW CITIES WITHIN MY IDEAL TEMPS + HAVE A HOTEL
#Personally I find this a more valuable display of data by only showing cities that have a hotel within my temp range.  
#Who wants to see potential cities you cannot stay at?

#This was not asked from us for the assignment but I wanted to see I could do it.

only_with_hotels_df = hotel_df[hotel_df['Hotel Name'] != 'No hotel found']

# Configure the map plot
map_plot_3 = only_with_hotels_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    hover_cols=['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name'],
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    scale = 0.5,
    color = "City"
)
# Display the map
map_plot_3


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)